In [1]:
import yfinance as yf
import talib
import numpy as np
import pandas as pd

In [2]:
# ============================================================================
# DOWNLOAD STOCK DATA FROM 2018 USING YFINANCE
# ============================================================================

# Configuration - Change these variables as needed
TICKER = 'QQQ'  # Any ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
START_DATE = '2018-01-01'  # Any start date in YYYY-MM-DD format

# Download data from start date onwards
stock_data = yf.download(TICKER, start=START_DATE, interval='1d')

if not stock_data.empty:
    print(f"✅ Successfully downloaded {len(stock_data)} records for {TICKER} from {START_DATE}")
    print(f"Data range: {stock_data.index.min().date()} to {stock_data.index.max().date()}")
    print("\nFirst 5 rows:")
    print(stock_data.head())
else:
    print(f"❌ Failed to download {TICKER} data from yfinance")

# Display the downloaded data
stock_data

/var/folders/tc/1rlgym492m70rw7ddhrpc1_80000gn/T/ipykernel_34572/2431403345.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(TICKER, start=START_DATE, interval='1d')
[*********************100%***********************]  1 of 1 completed

✅ Successfully downloaded 1944 records for QQQ from 2018-01-01
Data range: 2018-01-02 to 2025-09-25

First 5 rows:
Price            Close        High         Low        Open    Volume
Ticker             QQQ         QQQ         QQQ         QQQ       QQQ
Date                                                                
2018-01-02  150.605331  150.643335  148.400741  148.771339  32573300
2018-01-03  152.068695  152.201730  150.719340  150.747847  29383600
2018-01-04  152.334839  152.790956  152.116285  152.591411  24776100
2018-01-05  153.864731  153.969259  152.771947  153.057026  26992300
2018-01-08  154.463409  154.539431  153.807733  153.864746  23159100


Price,Close,High,Low,Open,Volume
Ticker,QQQ,QQQ,QQQ,QQQ,QQQ
Date,,,,,
2018-01-02,150.605331,150.643335,148.400741,148.771339,32573300
2018-01-03,152.068695,152.201730,150.719340,150.747847,29383600
2018-01-04,152.334839,152.790956,152.116285,152.591411,24776100
2018-01-05,153.864731,153.969259,152.771947,153.057026,26992300
2018-01-08,154.463409,154.539431,153.807733,153.864746,23159100
...,...,...,...,...,...
2025-09-19,598.655945,599.355146,595.159998,596.638325,58196100
2025-09-22,602.200012,602.869995,597.719971,597.739990,57154800


In [3]:
# ============================================================================
# TECHNICAL ANALYSIS INDICATORS USING TA-LIB
# ============================================================================

# Make sure stock_data is available from the previous cell
if "stock_data" not in locals():
    raise ValueError("Please run the stock data download cell first")

# Extract OHLCV data (handling multi-level columns from yfinance)
if isinstance(stock_data.columns, pd.MultiIndex):
    close = stock_data[("Close", TICKER)].values
    high = stock_data[("High", TICKER)].values
    low = stock_data[("Low", TICKER)].values
    open_ = stock_data[("Open", TICKER)].values
    volume = stock_data[("Volume", TICKER)].values
else:
    close = stock_data["Close"].values
    high = stock_data["High"].values
    low = stock_data["Low"].values
    open_ = stock_data["Open"].values
    volume = stock_data["Volume"].values

print(f"Calculating technical indicators for {TICKER}...")

# Simple Moving Averages
sma_20 = talib.SMA(close, timeperiod=20)
sma_50 = talib.SMA(close, timeperiod=50)

# Exponential Moving Averages
ema_12 = talib.EMA(close, timeperiod=12)
ema_26 = talib.EMA(close, timeperiod=26)

# MACD
macd, macdsignal, macdhist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)

# RSI
rsi = talib.RSI(close, timeperiod=14)

# Stochastic RSI
stochrsi_k, stochrsi_d = talib.STOCHRSI(close, timeperiod=14, fastk_period=3, fastd_period=3, fastd_matype=0)

# VWAP (manual calculation)
typical_price = (high + low + close) / 3
price_volume = typical_price * volume
cumulative_price_volume = np.cumsum(price_volume)
cumulative_volume = np.cumsum(volume)
vwap = cumulative_price_volume / cumulative_volume

# Momentum Indicator (using TA-Lib)
momentum_10 = talib.MOM(close, timeperiod=10)  # 10-period momentum
momentum_20 = talib.MOM(close, timeperiod=20)  # 20-period momentum

# Schaff Trend Cycle
cycle_period = 10
macd_cycle = talib.EMA(macd, timeperiod=cycle_period)
macd_smooth = talib.EMA(macd_cycle, timeperiod=cycle_period)
highest_macd = talib.MAX(macd_smooth, timeperiod=cycle_period)
lowest_macd = talib.MIN(macd_smooth, timeperiod=cycle_period)
stc_k = 100 * ((macd_smooth - lowest_macd) / (highest_macd - lowest_macd))
stc_d = talib.SMA(stc_k, timeperiod=3)

# Create indicators dataframe
indicators_df = pd.DataFrame({
    "Date": stock_data.index,
    "Close": close,
    "SMA_20": sma_20,
    "SMA_50": sma_50,
    "EMA_12": ema_12,
    "EMA_26": ema_26,
    "MACD": macd,
    "MACD_Signal": macdsignal,
    "MACD_Hist": macdhist,
    "RSI": rsi,
    "StochRSI_K": stochrsi_k,
    "StochRSI_D": stochrsi_d,
    "VWAP": vwap,
    "Momentum_10": momentum_10,
    "Momentum_20": momentum_20,
    "STC_K": stc_k,
    "STC_D": stc_d
})

print("✅ All technical indicators calculated!")
print(f"Data shape: {indicators_df.shape}")
indicators_df.tail(5)

Calculating technical indicators for QQQ...
✅ All technical indicators calculated!
Data shape: (1944, 15)


,Date,Close,SMA_20,SMA_50,EMA_12,EMA_26,MACD,MACD_Signal,MACD_Hist,RSI,StochRSI_K,StochRSI_D,VWAP,STC_K,STC_D
1939,2025-09-19,598.655945,579.388327,571.200223,586.578571,579.444460,7.134111,5.430886,1.703225,73.689960,100.0,6.666667e+01,310.359707,100.0,65.904728
1940,2025-09-22,602.200012,580.932944,572.173058,588.981869,581.130056,7.851813,5.915071,1.936742,75.762658,100.0,1.000000e+02,310.544223,100.0,89.528342
1941,2025-09-23,598.200012,582.359964,573.025739,590.400045,582.394498,8.005548,6.333166,1.672381,69.142027,0.0,6.666667e+01,310.751402,100.0,100.000000
1942,2025-09-24,596.099976,583.567615,573.826232,591.276957,583.409718,7.867239,6.639981,1.227258,65.886708,0.0,3.333333e+01,310.909439,100.0,100.000000
1943,2025-09-25,593.530029,584.602820,574.563939,591.623584,584.159371,7.464213,6.804827,0.659386,62.037364,0.0,7.579123e-14,311.130490,100.0,100.000000


In [4]:
# ============================================================================
# DATA SPLITTING FOR TRAINING AND VALIDATION (vectorbt-ready)
# ============================================================================

import vectorbt as vbt

# Config
TRAIN_RATIO = 0.7  # 70% train / 30% validation

# Compute split
total_days = len(stock_data)
train_size = int(total_days * TRAIN_RATIO)

train_data = stock_data.iloc[:train_size].copy()
val_data   = stock_data.iloc[train_size:].copy()

print("📊 Data Splitting Summary")
print(f"Total: {total_days}  |  Train: {len(train_data)}  |  Val: {len(val_data)}")
print(f"Train: {train_data.index[0].date()} → {train_data.index[-1].date()}")
print(f"Val  : {val_data.index[0].date()} → {val_data.index[-1].date()}")

def _select_close_series(df, ticker):
    if isinstance(df.columns, pd.MultiIndex):
        # 1) Exact match ('Close', TICKER)
        if ('Close', ticker) in df.columns:
            return df[('Close', ticker)].astype(float).rename(ticker)
        # 2) Any tuple containing 'Close'
        for col in df.columns:
            if isinstance(col, tuple) and ('Close' in col):
                return df[col].astype(float).rename(ticker)
        # 3) String contains 'Close'
        for col in df.columns:
            if 'Close' in str(col):
                return df[col].astype(float).rename(ticker)
        raise KeyError("Could not find a 'Close' column in MultiIndex columns")
    else:
        return df['Close'].astype(float).rename(ticker)

# Build clean Close series for vectorbt
train_close = _select_close_series(train_data, TICKER)
val_close   = _select_close_series(val_data, TICKER)

# Quick sanity
print("\nTraining Set Price Range:")
print(f"Start: ${train_close.iloc[0]:.2f}  End: ${train_close.iloc[-1]:.2f}  "
      f"Return: {(train_close.iloc[-1]/train_close.iloc[0]-1):.2%}")

print("\nValidation Set Price Range:")
print(f"Start: ${val_close.iloc[0]:.2f}  End: ${val_close.iloc[-1]:.2f}  "
      f"Return: {(val_close.iloc[-1]/val_close.iloc[0]-1):.2%}")

print("\n✅ Outputs ready: 'train_close' and 'val_close' (Pandas Series) for vectorbt.")

📊 Data Splitting Summary
Total: 1944  |  Train: 1360  |  Val: 584
Train: 2018-01-02 → 2023-05-26
Val  : 2023-05-30 → 2025-09-25

Training Set Price Range:
Start: $150.61  End: $343.20  Return: 127.88%

Validation Set Price Range:
Start: $344.76  End: $593.53  Return: 72.16%

✅ Outputs ready: 'train_close' and 'val_close' (Pandas Series) for vectorbt.


# 🚀 STOCHASTIC RSI GRID SEARCH - TRAINING SET

This section performs a comprehensive grid search optimization for the **Stochastic RSI Strategy** using only the **training data**.

The goal is to find the optimal overbought/oversold threshold combination that maximizes the Sharpe ratio on unseen data.

---

In [5]:
# Step 1: Define Parameter Ranges for StochRSI Strategy

# Overbought levels (sell signals when StochRSI_K crosses above these levels)
overbought_levels = [70, 75, 80, 85, 90]
print("📈 Overbought Levels (sell signals):")
for i, level in enumerate(overbought_levels, 1):
    print(f"  {i}. {level}%")

# Oversold levels (buy signals when StochRSI_K crosses below these levels)
oversold_levels = [10, 15, 20, 25, 30]
print("📉 Oversold Levels (buy signals):")
for i, level in enumerate(oversold_levels, 1):
    print(f"  {i}. {level}%")

# Generate all valid combinations (oversold < overbought)
stochrsi_combinations = []
for oversold in oversold_levels:
    for overbought in overbought_levels:
        if oversold < overbought:
            stochrsi_combinations.append((oversold, overbought))

print(f"✅ Generated {len(stochrsi_combinations)} valid StochRSI combinations")
print("\n📋 All combinations preview:")
for i, (oversold, overbought) in enumerate(stochrsi_combinations, 1):
    print(f"  {i:2d}. Oversold: {oversold:2d}% | Overbought: {overbought:2d}%")

print("\n🎯 Ready to test all combinations on training data!")

📈 Fast SMA Periods (short-term):
  1. 5 periods
  2. 10 periods
  3. 15 periods
  4. 20 periods
  5. 25 periods
  6. 30 periods
📉 Slow SMA Periods (long-term):
  1. 30 periods
  2. 40 periods
  3. 50 periods
  4. 60 periods
  5. 80 periods
  6. 100 periods
✅ Generated 35 valid MA combinations

📋 First 10 combinations preview:
   1. Fast:  5 | Slow: 30
   2. Fast:  5 | Slow: 40
   3. Fast:  5 | Slow: 50
   4. Fast:  5 | Slow: 60
   5. Fast:  5 | Slow: 80
   6. Fast:  5 | Slow: 100
   7. Fast: 10 | Slow: 30
   8. Fast: 10 | Slow: 40
   9. Fast: 10 | Slow: 50
  10. Fast: 10 | Slow: 60
   ... and 25 more combinations

🎯 Ready to test all combinations on training data!


In [6]:
# Step 2: Initialize Results Collection System

# Create empty list to store all backtest results
grid_search_results = []

print("📊 Results Collection System Initialized")
print(f"   - Will test {len(stochrsi_combinations)} StochRSI combinations")
print("   - Results will be stored in 'grid_search_results' list")

# Define what metrics we will collect
# Define what metrics we will collect (All TradingView-style metrics)
metrics_to_collect = [
    # Strategy Parameters
    "oversold_level",
    "overbought_level",
    
    # Return Metrics
    "total_return",
    "annualized_return",
    "total_profit",
    
    # Risk-Adjusted Return Metrics
    "sharpe_ratio",
    "sortino_ratio",
    "calmar_ratio",
    "omega_ratio",
    "information_ratio",
    "tail_ratio",
    "deflated_sharpe_ratio",
    
    # Risk Metrics
    "max_drawdown",
    "volatility",
    "ulcer_index",
    
    # Trade Performance Metrics
    "win_rate",
    "total_trades",
    "avg_trade_duration",
    "expectancy",
    "profit_factor",
    "sqn",
    
    # Win/Loss Analysis
    "payoff_ratio",
    "largest_win",
    "largest_loss",
    "avg_win_amount",
    "avg_loss_amount",
    "winning_streak",
    "losing_streak",
    
    # Additional Ratios
    "recovery_factor",
    "gain_to_pain_ratio",
    "serenity_index"
]

print("📈 Metrics to collect for each combination:")
for i, metric in enumerate(metrics_to_collect, 1):
    print(f"  {i}. {metric.replace('_', ' ').title()}")

print("\n🚀 Ready to start the StochRSI grid search!")

📊 Results Collection System Initialized
   - Will test 35 MA combinations
   - Each test will generate 6 performance metrics
   - Results will be stored in 'grid_search_results' list
📈 Metrics to collect for each combination:
  1. Fast Period
  2. Slow Period
  3. Total Return
  4. Sharpe Ratio
  5. Max Drawdown
  6. Win Rate
  7. Total Trades

🚀 Ready to start the grid search!


In [8]:

# STEP 3: EXECUTE COMPREHENSIVE MA CROSSOVER GRID SEARCH ON TRAINING DATA


import vectorbt as vbt

print("🔬 INITIATING MA CROSSOVER GRID SEARCH OPTIMIZATION")
print("=" * 70)
print(f"📊 Testing Strategy: Simple Moving Average Crossover")
print(f"📅 Training Period: {train_close.index[0].date()} → {train_close.index[-1].date()}")
print(f"💰 Initial Capital: $100,000")
print(f"💸 Transaction Costs: 0.05% per trade (fees + slippage)")
print(f"🎯 Optimization Metric: Sharpe Ratio (risk-adjusted returns)")
print("=" * 70)

# Initialize grid search tracking
total_combinations = len(ma_combinations)
completed_tests = 0
successful_tests = 0
failed_tests = 0

print(f"🚀 Starting grid search across {total_combinations} parameter combinations...")
print()

# MAIN GRID SEARCH LOOP
for i, (fast_period, slow_period) in enumerate(ma_combinations, 1):
    
    try:
        # ==========================================
        # INDICATOR CALCULATION
        # ==========================================
        
        # Calculate Simple Moving Averages on training data
        fast_ma = vbt.MA.run(train_close, fast_period, ewm=False)  # Simple MA
        slow_ma = vbt.MA.run(train_close, slow_period, ewm=False)  # Simple MA
        
        # ==========================================
        # SIGNAL GENERATION
        # ==========================================
        
        # BUY SIGNAL: Fast MA crosses above Slow MA (Golden Cross)
        entries = fast_ma.ma_crossed_above(slow_ma.ma)
        
        # SELL SIGNAL: Fast MA crosses below Slow MA (Death Cross)  
        exits = fast_ma.ma_crossed_below(slow_ma.ma)
        
        # ==========================================
        # BACKTEST EXECUTION
        # ==========================================
        
        # Run portfolio simulation with realistic trading costs
        portfolio = vbt.Portfolio.from_signals(
            close=train_close,           # Price data
            entries=entries,             # Buy signals
            exits=exits,                 # Sell signals
            init_cash=100_000,           # Starting capital
            fees=0.0005,                 # 5 basis points commission
            slippage=0.0005,             # 5 basis points slippage
            freq='D'                     # Daily frequency for calculations
        )
        
        # ==========================================
        # PERFORMANCE METRICS CALCULATION
        # ==========================================
        
        # ==========================================
        # COMPREHENSIVE PERFORMANCE METRICS CALCULATION
        # ==========================================
        
        # Return Metrics
        total_return = portfolio.total_return()
        annualized_return = portfolio.annualized_return(freq='D')
        total_profit = portfolio.total_profit()
        
        # Risk-Adjusted Return Metrics (TradingView style)
        sharpe_ratio = portfolio.sharpe_ratio(freq='D')
        sortino_ratio = portfolio.sortino_ratio(freq='D')
        calmar_ratio = portfolio.calmar_ratio(freq='D')
        omega_ratio = portfolio.omega_ratio(freq='D')
        information_ratio = portfolio.information_ratio(freq='D')
        tail_ratio = portfolio.tail_ratio(freq='D')
        deflated_sharpe_ratio = portfolio.deflated_sharpe_ratio(freq='D')
        
        # Risk Metrics
        max_drawdown = portfolio.max_drawdown()
        volatility = portfolio.annualized_volatility(freq='D')
        
        # Calculate Ulcer Index (custom calculation)
        returns = portfolio.returns()
        cumulative_returns = (1 + returns).cumprod()
        peak = cumulative_returns.expanding().max()
        drawdowns = (cumulative_returns - peak) / peak
        ulcer_index = np.sqrt((drawdowns ** 2).mean()) if len(drawdowns) > 0 else 0
        
        # Trade Performance Metrics
        win_rate = portfolio.trades.win_rate()
        total_trades = len(portfolio.trades)
        avg_trade_duration = portfolio.trades.duration.mean()
        expectancy = portfolio.trades.expectancy()
        profit_factor = portfolio.trades.profit_factor()
        sqn = portfolio.trades.sqn()
        
        # Win/Loss Analysis
        trade_returns = portfolio.trades.returns
        winning_trades = trade_returns[trade_returns > 0]
        losing_trades = trade_returns[trade_returns < 0]
        
        # Payoff Ratio (Average Win / Average Loss)
        avg_win_amount = winning_trades.mean() if len(winning_trades) > 0 else 0
        avg_loss_amount = abs(losing_trades.mean()) if len(losing_trades) > 0 else 0
        payoff_ratio = avg_win_amount / avg_loss_amount if avg_loss_amount != 0 else np.inf
        
        # Largest Win/Loss
        largest_win = winning_trades.max() if len(winning_trades) > 0 else 0
        largest_loss = losing_trades.min() if len(losing_trades) > 0 else 0
        
        # Streak Analysis
        winning_streak = portfolio.trades.winning_streak()
        losing_streak = portfolio.trades.losing_streak()
        
        # Additional Ratios
        recovery_factor = total_return / abs(max_drawdown) if max_drawdown != 0 else np.inf
        
        # Gain to Pain Ratio (Total Return / Sum of Negative Returns)
        negative_returns = returns[returns < 0]
        gain_to_pain_ratio = total_return / abs(negative_returns.sum()) if negative_returns.sum() != 0 else np.inf
        
        # Serenity Index (TradingView style: (Total Return + 1) / (Max Drawdown^2 + 1))
        serenity_index = (total_return + 1) / (max_drawdown**2 + 1) if max_drawdown != 0 else total_return + 1
        
        # ==========================================
        # RESULTS STORAGE
        # ==========================================
        
        result = {
            # Strategy Parameters
            "fast_period": fast_period,
            "slow_period": slow_period,
            
            # Return Metrics
            "total_return": total_return,
            "annualized_return": annualized_return,
            "total_profit": total_profit,
            
            # Risk-Adjusted Return Metrics
            "sharpe_ratio": sharpe_ratio,
            "sortino_ratio": sortino_ratio,
            "calmar_ratio": calmar_ratio,
            "omega_ratio": omega_ratio,
            "information_ratio": information_ratio,
            "tail_ratio": tail_ratio,
            "deflated_sharpe_ratio": deflated_sharpe_ratio,
            
            # Risk Metrics
            "max_drawdown": max_drawdown,
            "volatility": volatility,
            "ulcer_index": ulcer_index,
            
            # Trade Performance Metrics
            "win_rate": win_rate,
            "total_trades": total_trades,
            "avg_trade_duration": avg_trade_duration,
            "expectancy": expectancy,
            "profit_factor": profit_factor,
            "sqn": sqn,
            
            # Win/Loss Analysis
            "payoff_ratio": payoff_ratio,
            "largest_win": largest_win,
            "largest_loss": largest_loss,
            "avg_win_amount": avg_win_amount,
            "avg_loss_amount": avg_loss_amount,
            "winning_streak": winning_streak,
            "losing_streak": losing_streak,
            
            # Additional Ratios
            "recovery_factor": recovery_factor,
            "gain_to_pain_ratio": gain_to_pain_ratio,
            "serenity_index": serenity_index
        }
        
        grid_search_results.append(result)
        successful_tests += 1
        
        # ==========================================
        # PROGRESS REPORTING
        # ==========================================
        
        # Show progress every 10 combinations or on last one
        if i % 10 == 0 or i == total_combinations:
            progress_pct = (i / total_combinations) * 100
            print(f"📈 Progress: {i}/{total_combinations} combinations tested ({progress_pct:.1f}%)")
            print(f"   ✅ Successful: {successful_tests} | ❌ Failed: {failed_tests}")
            print(f"   📊 Latest: SMA({fast_period},{slow_period}) → Sharpe: {sharpe_ratio:.3f}, Return: {total_return:.1%}")
            print(f"      Trades: {total_trades}, Win Rate: {win_rate:.1%}, Max DD: {max_drawdown:.1%}")
            print()
            
    except Exception as e:
        failed_tests += 1
        if i % 10 == 0 or i == total_combinations:
            print(f"⚠️  Error with SMA({fast_period},{slow_period}): {str(e)[:50]}...")
        continue

# ==========================================
# GRID SEARCH COMPLETION SUMMARY
# ==========================================

print("=" * 70)
print("🎯 GRID SEARCH COMPLETED!")
print("=" * 70)
print(f"📊 Total combinations attempted: {total_combinations}")
print(f"✅ Successfully completed: {successful_tests}")
print(f"❌ Failed: {failed_tests}")
print(f"📈 Success rate: {(successful_tests/total_combinations)*100:.1f}%")
print()
print("📋 Results Summary:")
print(f"   - Results stored in 'grid_search_results' list")
print(f"   - Each result contains {len(metrics_to_collect)} comprehensive performance metrics")
print(f"   - Data ready for analysis in next step")
print("=" * 70)

🔬 INITIATING MA CROSSOVER GRID SEARCH OPTIMIZATION
📊 Testing Strategy: Simple Moving Average Crossover
📅 Training Period: 2018-01-02 → 2023-05-26
💰 Initial Capital: $100,000
💸 Transaction Costs: 0.05% per trade (fees + slippage)
🎯 Optimization Metric: Sharpe Ratio (risk-adjusted returns)
🚀 Starting grid search across 35 parameter combinations...

📈 Progress: 10/35 combinations tested (28.6%)
   ✅ Successful: 10 | ❌ Failed: 0
   📊 Latest: SMA(10,60) → Sharpe: 0.884, Return: 73.9%
      Trades: 15, Win Rate: 60.0%, Max DD: -16.3%

📈 Progress: 20/35 combinations tested (57.1%)
   ✅ Successful: 20 | ❌ Failed: 0
   📊 Latest: SMA(20,40) → Sharpe: 0.540, Return: 39.0%
      Trades: 17, Win Rate: 58.8%, Max DD: -26.8%

📈 Progress: 30/35 combinations tested (85.7%)
   ✅ Successful: 30 | ❌ Failed: 0
   📊 Latest: SMA(25,100) → Sharpe: 0.558, Return: 43.9%
      Trades: 5, Win Rate: 60.0%, Max DD: -34.5%

📈 Progress: 35/35 combinations tested (100.0%)
   ✅ Successful: 35 | ❌ Failed: 0
   📊 Latest:

In [ ]:
# Step 4: Analyze Grid Search Results

import pandas as pd

# Convert results to DataFrame for analysis
results_df = pd.DataFrame(grid_search_results)

print("📊 Grid Search Results Analysis")
print("=" * 50)
print(f"Total combinations tested: {len(results_df)}")
print(f"Results shape: {results_df.shape}")

# Comprehensive TradingView-style statistics
print("📈 Comprehensive Performance Statistics:")
print("-" * 50)

# Return Metrics
print("💰 Return Metrics:")
print(f"   Best Total Return: {results_df['total_return'].max():.2%}")
print(f"   Average Total Return: {results_df['total_return'].mean():.2%}")
print(f"   Best Annualized Return: {results_df['annualized_return'].max():.2%}")

# Risk-Adjusted Metrics
print("📊 Risk-Adjusted Return Metrics:")
print(f"   Best Sharpe Ratio: {results_df['sharpe_ratio'].max():.3f}")
print(f"   Best Sortino Ratio: {results_df['sortino_ratio'].max():.3f}")
print(f"   Best Calmar Ratio: {results_df['calmar_ratio'].max():.3f}")
print(f"   Best Omega Ratio: {results_df['omega_ratio'].max():.3f}")

# Risk Metrics
print("⚠️  Risk Metrics:")
print(f"   Average Max Drawdown: {results_df['max_drawdown'].mean():.2%}")
print(f"   Best Max Drawdown: {results_df['max_drawdown'].min():.2%}")
print(f"   Average Volatility: {results_df['volatility'].mean():.2%}")

# Trade Performance
print("🎯 Trade Performance:")
print(f"   Best Win Rate: {results_df['win_rate'].max():.1%}")
print(f"   Average Win Rate: {results_df['win_rate'].mean():.1%}")
print(f"   Best Profit Factor: {results_df['profit_factor'].max():.2f}")
print(f"   Total Trades Range: {results_df['total_trades'].min()} - {results_df['total_trades'].max()}")

# Additional TradingView Metrics
print("🔬 Advanced Metrics:")
print(f"   Best Recovery Factor: {results_df['recovery_factor'].max():.2f}")
print(f"   Best Payoff Ratio: {results_df['payoff_ratio'].max():.2f}")
print(f"   Best SQN: {results_df['sqn'].max():.2f}")
print(f"   Highest Serenity Index: {results_df['serenity_index'].max():.2f}")

# Find best combination by Sharpe ratio
best_result = results_df.loc[results_df['sharpe_ratio'].idxmax()]

print("🏆 BEST MA CROSSOVER STRATEGY (by Sharpe Ratio)")
print("-" * 50)
print(f"Fast SMA Period: {int(best_result['fast_period'])}")
print(f"Slow SMA Period: {int(best_result['slow_period'])}")
print(f"Total Return: {best_result['total_return']:.2%}")
print(f"Sharpe Ratio: {best_result['sharpe_ratio']:.3f}")
print(f"Max Drawdown: {best_result['max_drawdown']:.2%}")
print(f"Win Rate: {best_result['win_rate']:.1%}")
print(f"Total Trades: {int(best_result['total_trades'])}")

print("\n📋 Top 5 Best Performers:")
top_5 = results_df.nlargest(5, "sharpe_ratio")
for i, (_, row) in enumerate(top_5.iterrows(), 1):
    print(f"{i}. SMA({int(row['fast_period'])},{int(row['slow_period'])}) → Sharpe: {row['sharpe_ratio']:.3f}, Return: {row['total_return']:.1%}")

print("\n✅ Analysis complete! Best strategy identified.")

In [ ]:
# Step 5: Validate Best Strategy on Validation Set

print("🔍 Validating Best Strategy on Unseen Validation Data")
print("=" * 60)

# Extract best parameters from training
best_fast = int(best_result['fast_period'])
best_slow = int(best_result['slow_period'])

print(f"Testing best training strategy: SMA({best_fast}, {best_slow})")
print(f"Validation period: {val_close.index[0].date()} to {val_close.index[-1].date()}")

# Apply best strategy to validation data
val_fast_ma = vbt.MA.run(val_close, best_fast, ewm=False)
val_slow_ma = vbt.MA.run(val_close, best_slow, ewm=False)

# Generate signals on validation data
val_entries = val_fast_ma.ma_crossed_above(val_slow_ma.ma)
val_exits = val_fast_ma.ma_crossed_below(val_slow_ma.ma)

# Run validation backtest
val_portfolio = vbt.Portfolio.from_signals(
    val_close,
    val_entries,
    val_exits,
    init_cash=100_000,
    fees=0.0005,
    slippage=0.0005
)

# Calculate validation metrics
val_total_return = val_portfolio.total_return()
val_sharpe_ratio = val_portfolio.sharpe_ratio()
val_max_drawdown = val_portfolio.max_drawdown()
val_win_rate = val_portfolio.trades.win_rate()
val_total_trades = len(val_portfolio.trades)

print("📊 VALIDATION RESULTS")
print("-" * 30)
print(f"Strategy: SMA({best_fast}, {best_slow})")
print(f"Total Return: {val_total_return:.2%}")
print(f"Sharpe Ratio: {val_sharpe_ratio:.3f}")
print(f"Max Drawdown: {val_max_drawdown:.2%}")
print(f"Win Rate: {val_win_rate:.1%}")
print(f"Total Trades: {val_total_trades}")

print("\n🔄 TRAINING vs VALIDATION COMPARISON")
print("-" * 40)
print(f"Total Return:  Train: {best_result['total_return']:.2%}  |  Val: {val_total_return:.2%}")
print(f"Sharpe Ratio:   Train: {best_result['sharpe_ratio']:.3f}  |  Val: {val_sharpe_ratio:.3f}")
print(f"Max Drawdown:  Train: {best_result['max_drawdown']:.2%}  |  Val: {val_max_drawdown:.2%}")
print(f"Win Rate:      Train: {best_result['win_rate']:.1%}  |  Val: {val_win_rate:.1%}")
print(f"Total Trades:  Train: {best_result['total_trades']}  |  Val: {val_total_trades}")

print("\n🎯 CONCLUSION: Strategy validated on unseen data!")
if val_sharpe_ratio > 0.5:
    print("✅ Strategy shows promising risk-adjusted returns")
elif val_sharpe_ratio > 0:
    print("🤔 Strategy has positive but modest risk-adjusted returns")
else:
    print("❌ Strategy underperforms buy-and-hold on validation data")